# Food in Art

### Angles

- Proportion of food related items in museums
- Proportion of food related artworks by epoch and country
- Is food central topic
- Network diagram linking culinary styles with corresponding art movements.
- fruit, meal, dessert, etc
- feasts, banquets, etc
- food as a medium?
- food symbolism?
- **Analysis Angle:** Investigate how the introduction of new food items through trade influenced their depiction in art.
    - **Visualization:** Maps showing trade routes alongside corresponding changes in food representation in art."
- compare current food focus by country
- Nutrional evolution
- **Analysis Angle:** Explore how periods of famine or abundance influenced the portrayal of food in art.
    - **Visualization:** Correlation charts linking historical events of scarcity or abundance with changes in food depiction."
- Social classes?
- Agriculture, markets?
- **Analysis Angle:** Analyze the use of color in food-themed art and its psychological effects on viewers.
- **Visualization:** Color wheel charts displaying dominant hues in various artworks."
- **Analysis Angle:** Investigate how chefs create dishes inspired by artworks, merging visual art with gastronomy.
- **Visualization:** Side-by-side images of artworks and the culinary dishes they inspired."